# Kickstarter and SQL `10 points`

Since I claimed that the point of Lede was to be able to learn anything using Google, I should probably test that, too, yeah?

Working with **SQL databases** is something we didn't learn! It comes in a lot of formats - PostreSQL, MySQL, Sqlite - but they are all generally the same thing.

SQL is great for when you're reading in big big big datasets: instead of waiting an hour for `.read_csv` to crash on a giant file that you're going to filter anyway, instead you just ask SQL to give you a subset of the data to work on.

**Topics**

* Learning new things with the power of the internet
* SQL

### The data

**Kickstarter** is a website people use to raise money for projects. It used to be really popular, but it's since died down a bit. We're going to work with [a dataset about Kickstarter in SQLite format](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/EOYBXM).

You'll want to download the database by going to **Access Dataset** and then **Download ZIP**.

It's a big file, and it's compressed! We'll be using the `kickstarter.db` file that is inside the zip file: when you open up the zip, I extracted it using [The Unarchiver](https://theunarchiver.com/) by telling it to open up the `.z01` file (it's so big it's spread across both the `.zip` and the `.z01`). 

## Open the dataset

Just like an Excel file can have different sheets, a SQL database has different tables.

In [2]:
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option('display.float_format', '{:,.2f}'.format)

SQLite + pandas doesn't use the "normal" way to ask for tables, so I'll give that line to you.

In [3]:
import sqlite3

# Open a connection
conn = sqlite3.connect("files/kickstarter.db")

# Make a query, save it into a dataframe
df = pd.read_sql_query('SELECT name FROM sqlite_master WHERE type = "table";', conn)

# Close the connection
conn.close()

# See the results in the dataframe
df

,name
0,urls_to_scrape
1,all_files
2,category
3,creator
4,funding_trend
5,item
6,livestream
7,location
8,project
9,reward


## Simple selects `3 points`

### Select all of the data for the first 4 kickstarter projects

They live in the `project` table. This will help you understand how SQL queries work.

In [12]:
def dbquery(query):
    conn = sqlite3.connect("files/kickstarter.db")
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

In [13]:
dbquery('SELECT * FROM project ORDER BY created_at ASC LIMIT 4;')

,id,state,url_project,url_project_short,name,country,creator_id,location_id,category_id,created_at,deadline,updated_at,state_changed_at,successful_at,launched_at,goal,pledged,currency,currency_symbol,usd_pledged,static_usd_rate,backers_count,comments_count,updates_count,spotlight,staff_pick,blurb,currency_trailing_code,disable_communication,photo_url,profile_background_color,profile_background_image_opacity,profile_blurb,profile_id,profile_link_background_color,profile_link_text,profile_link_text_color,profile_link_url,profile_name,profile_project_id,profile_should_show_feature_image_section,profile_show_feature_image,profile_state,profile_state_changed_at,profile_text_color,slug,url_rewards,url_updates,video_id,video_url_high,video_url_webm,video_height,video_width,video_status,file_name,last_modification,deleted_comments
0,1860890148,failed,https://www.kickstarter.com/projects/perry/gra...,http://kck.st/aRzK5V,Grace Jones Does Not Give A F$#% T-Shirt (limi...,US,4,"12,589,335.00",9,1240333219,1243753199,1397849643,1243753227,NaN,1240340568,1000,625.00,USD,$,625.00,1,30,9,2,False,False,About two years ago I realized a profound trut...,True,False,https://ksr-ugc.imgix.net/assets/011/257/331/e...,None,0.80,None,59,None,None,None,None,None,59,True,False,inactive,1425915800,None,grace-jones-does-not-give-a-f-t-shirt-limited-...,https://www.kickstarter.com/projects/perry/gra...,https://www.kickstarter.com/projects/perry/gra...,1524,https://ksr-video.imgix.net/projects/3/video-1...,None,420,560,successful,www.kickstarter.com/projects/perry/grace-jones...,2018-05-27 00:19:54,0.00
1,1473103893,successful,https://www.kickstarter.com/projects/williambr...,http://kck.st/caFA3s,TELEVISION LANDSCAPE: Making the Record.,US,1505954783,NaN,40,1240335335,1253030340,1479099902,1253030408,"1,253,030,408.00",1245374186,2000,"2,000.00",USD,$,"2,000.00",1,33,0,0,True,False,"William Brittelle's ""Television Landscape"" is ...",True,False,https://ksr-ugc.imgix.net/assets/011/257/333/2...,None,0.80,None,61,None,None,None,None,None,61,True,False,inactive,1425915800,None,television-landscape-making-the-record,https://www.kickstarter.com/projects/williambr...,https://www.kickstarter.com/projects/williambr...,1614,https://ksr-video.imgix.net/projects/5/video-1...,None,420,560,successful,www.kickstarter.com/projects/williambrittelle/...,2017-03-11 16:11:28,NaN
2,709707365,failed,https://www.kickstarter.com/projects/videothin...,http://kck.st/dymfMD,CRYSTAL ANTLERS UNTITLED MOVIE,US,1292964752,"2,442,047.00",32,1240356396,1248073199,1397849644,1248073228,NaN,1240438073,80000,22.00,USD,$,22.00,1,3,1,0,False,False,Above is a music video edited together with sh...,True,False,https://ksr-ugc.imgix.net/assets/011/257/334/1...,None,0.80,None,62,None,None,None,None,None,62,True,False,inactive,1425915800,None,crystal-antlers-untitled-movie,https://www.kickstarter.com/projects/videothin...,https://www.kickstarter.com/projects/videothin...,1527,https://ksr-video.imgix.net/projects/6/video-1...,None,316,560,successful,www.kickstarter.com/projects/videothing/crysta...,2018-05-25 00:30:55,0.00
3,405307432,successful,https://www.kickstarter.com/projects/Samplefre...,http://kck.st/cO8V5u,Electronola - An electronic gumbo of New Orlea...,US,11,"2,458,833.00",38,1240366270,1244185140,1397849645,1244185224,"1,244,185,224.00",1241192665,4000,"4,100.60",USD,$,"4,100.60",1,114,30,50,True,True,"UPDATE: Shannon Powell, Walter Payton, Lucien ...",True,False,https://ksr-ugc.imgix.net/assets/011/257/336/a...,None,0.80,None,63,None,None,None,None,None,63,True,False,inactive,1425915800,None,electronola-an-electronic-gumbo-of-new-orleans...,https://www.kickstarter.com/projects/Samplefre...,https://www.kickstarter.com/projects/Samplefre...,1534,https://ksr-video.imgix.net/projects/8/video-1...,None,420,560,successful,www.kickstarter.com/projects/Samplefreq/electr...,2018-05-24 15:49:19,0.00


### Select the top 10 projects with the highest amount pledged

I only want to see the name and the amount pledged. **The number one result should be the Pebble watch** - if your result is something Japanese, try querying all of the columns and figure out what went wrong.

In [15]:
dbquery('SELECT name,usd_pledged FROM project ORDER BY usd_pledged DESC LIMIT 10;')

,name,usd_pledged
0,"Pebble Time - Awesome Smartwatch, No Compromises","20,338,986.27"
1,COOLEST COOLER: 21st Century Cooler that's Act...,"13,285,226.36"
2,"Pebble 2, Time 2 + All-New Pebble Core","12,779,843.49"
3,Kingdom Death: Monster 1.5,"12,393,139.69"
4,Pebble: E-Paper Watch for iPhone and Android,"10,266,845.74"
5,The World's Best TRAVEL JACKET with 15 Feature...,"9,192,055.66"
6,Exploding Kittens,"8,782,571.99"
7,OUYA: A New Kind of Video Game Console,"8,596,474.58"
8,"THE 7th CONTINENT – What Goes Up, Must Come Down.","7,072,757.00"
9,"The Everyday Backpack, Tote, and Sling","6,565,782.50"


## Filtering your queries `3 points`

### Find me – yes, me, Soma! – in the project creators table

In [16]:
dbquery('SELECT * FROM creator WHERE name = "Jonathan Soma";')

,id,name,slug,avatar_thumb,avatar_small,urls_web_user,avatar_medium,is_registered,urls_api_user,chosen_currency
0,546599331,Jonathan Soma,soma,https://ksr-ugc.imgix.net/assets/005/801/114/d...,https://ksr-ugc.imgix.net/assets/005/801/114/d...,https://www.kickstarter.com/profile/soma,https://ksr-ugc.imgix.net/assets/005/801/114/d...,None,https://api.kickstarter.com/v1/users/546599331...,None


### Find all of my projects

In [17]:
dbquery('SELECT * FROM project WHERE creator_id = "546599331";')

,id,state,url_project,url_project_short,name,country,creator_id,location_id,category_id,created_at,deadline,updated_at,state_changed_at,successful_at,launched_at,goal,pledged,currency,currency_symbol,usd_pledged,static_usd_rate,backers_count,comments_count,updates_count,spotlight,staff_pick,blurb,currency_trailing_code,disable_communication,photo_url,profile_background_color,profile_background_image_opacity,profile_blurb,profile_id,profile_link_background_color,profile_link_text,profile_link_text_color,profile_link_url,profile_name,profile_project_id,profile_should_show_feature_image_section,profile_show_feature_image,profile_state,profile_state_changed_at,profile_text_color,slug,url_rewards,url_updates,video_id,video_url_high,video_url_webm,video_height,video_width,video_status,file_name,last_modification,deleted_comments
0,320516934,successful,https://www.kickstarter.com/projects/soma/broo...,http://kck.st/d9d9Sx,Brooklyn Brainery Needs a Home!,US,546599331,12589335,1,1270872639,1278129540,1397755136,1278129615,1278129615,1271182430,9500,"9,629.95",USD,$,"9,629.95",1,204,9,9,True,True,"We host awesomely cheap, collaborative classes...",True,False,https://ksr-ugc.imgix.net/assets/011/261/121/e...,None,0.80,None,4894,None,None,None,None,None,4894,True,False,inactive,1425915800,None,brooklyn-brainery-needs-a-home,https://www.kickstarter.com/projects/soma/broo...,https://www.kickstarter.com/projects/soma/broo...,"3,709.00",https://ksr-video.imgix.net/projects/4746/vide...,None,314.00,560.00,successful,www.kickstarter.com/projects/soma/brooklyn-bra...,2018-05-24 05:28:04,0
1,1052891075,successful,https://www.kickstarter.com/projects/soma/cook...,http://kck.st/aZ8hta,Cooking Class in a Box [Exotic Cuisine Edition],US,546599331,12589335,10,1276719823,1278991380,1397756801,1278991807,1278991807,1276792614,200,"1,820.44",USD,$,"1,820.44",1,53,8,7,True,True,I'll build you a cooking class on an exotic cu...,True,False,https://ksr-ugc.imgix.net/assets/011/263/591/4...,None,0.80,None,7857,None,None,None,None,None,7857,True,False,inactive,1425915800,None,cooking-class-in-a-box-exotic-cuisine-edition,https://www.kickstarter.com/projects/soma/cook...,https://www.kickstarter.com/projects/soma/cook...,NaN,None,None,NaN,NaN,None,www.kickstarter.com/projects/soma/cooking-clas...,2018-05-25 09:41:14,0
2,1606179513,successful,https://www.kickstarter.com/projects/soma/ice-...,http://kck.st/cgpgNy,Ice Cream Club,US,546599331,2459115,10,1271981825,1274055420,1397755379,1274056206,1274056206,1272034766,220,485.98,USD,$,485.98,1,27,7,5,True,True,When it's 100 degrees outside you'll wish you ...,True,False,https://ksr-ugc.imgix.net/assets/011/261/487/c...,None,0.80,None,5368,None,None,None,None,None,5368,True,False,inactive,1425915800,None,ice-cream-club,https://www.kickstarter.com/projects/soma/ice-...,https://www.kickstarter.com/projects/soma/ice-...,NaN,None,None,NaN,NaN,None,www.kickstarter.com/projects/soma/ice-cream-club,2018-05-27 00:19:54,0


## Calculations `3 points`

### Find how much money has been raised on Kickstarter by **successful** projects.

In [21]:
dbquery('SELECT SUM(usd_pledged) AS total FROM project WHERE state = "successful";')

,total
0,"3,643,252,201.72"


### Find how much money has been raised on Kickstarter by "successful" projects in **the United States**.

The `state` can be successful, failed, live, etc.

In [22]:
dbquery('SELECT SUM(usd_pledged) AS total FROM project WHERE state = "successful" AND country = "US";')

,total
0,"2,961,700,473.92"


## Aggregating `3 points`

## How many projects were successful/unsuccessful/etc?

**Use your SQL query to calculate the answer.** Your result should look something like this:

| |...|...|
|---|---|---|
|0|canceled|36659|
|1|failed|211783|
|2|live|3163|
|3|purged|202|
|4|successful|155130|
|5|suspended|1700|

In [24]:
dbquery('SELECT state, COUNT(*) AS total FROM project GROUP BY state;')

,state,total
0,canceled,36659
1,failed,211783
2,live,3163
3,purged,202
4,successful,155130
5,suspended,1700


### How much money has been pledged to campaigns in each country?

In [26]:
dbquery('SELECT country, SUM(usd_pledged) AS total FROM project GROUP BY country ORDER BY total DESC;')

,country,total
0,US,"3,270,883,676.02"
1,GB,"298,732,943.23"
2,CA,"112,432,907.01"
3,AU,"54,965,224.87"
4,DE,"54,424,475.45"
5,FR,"48,536,026.19"
6,NL,"36,543,906.82"
7,IT,"25,627,131.64"
8,ES,"24,281,129.46"
9,HK,"22,263,452.47"


## Conversion to CSV `2 points`

### Save the result of your last query - the one about amount raised - to a CSV

In [29]:
dbquery('SELECT country, SUM(usd_pledged) AS total FROM project GROUP BY country ORDER BY total DESC;').to_csv('ranking.csv')

### Visualize the results using DataWrapper, highlighting the country of your choice

Link in the cell below.

[https://datawrapper.dwcdn.net/qy6bY/3/](https://datawrapper.dwcdn.net/qy6bY/3/)

### Building on your last query, calculate the number of projects in each category `2 points`

In [39]:
projects = dbquery('SELECT category.name, COUNT(project.id) AS total \
    FROM project INNER JOIN category on project.category_id = category.id \
    GROUP BY project.category_id')

projects

,name,total
0,Art,8725
1,Comics,5194
2,Dance,2389
3,Design,3884
4,Fashion,8303
...,...,...
164,Makerspaces,265
165,Comedy,39
166,Comedy,134
167,Comedy,172


### Using pandas, sort the result to get the top 20 project categories `1 point`

In [40]:
projects.sort_values('total',ascending=False).head(20)

,name,total
22,Product Design,24270
9,Music,17055
28,Tabletop Games,16702
24,Documentary,15961
26,Shorts,12572
29,Video Games,12492
5,Food,11054
6,Film & Video,10523
41,Fiction,9510
0,Art,8725
